# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: make stuff finally work 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'F') & \
                             (sessions.full_genotype.str.find('Sst-IRES') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
760693773,2019-10-03T00:00:00Z,738651054,brain_observatory_1.1,110.0,F,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,826,2233,6,"[grey, VISrl, nan, VISal, VISp, VISpm, VISam]"


In [7]:
session_id = filtered_sessions.index.values[0]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 826


,amplitude_cutoff,waveform_recovery_slope,cluster_id,waveform_spread,waveform_velocity_below,d_prime,nn_miss_rate,isolation_distance,silhouette_score,waveform_halfwidth,waveform_PT_ratio,cumulative_drift,isi_violations,presence_ratio,snr,L_ratio,waveform_amplitude,waveform_repolarization_slope,local_index_unit,waveform_velocity_above,nn_hit_rate,peak_channel_id,waveform_duration,firing_rate,max_drift,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951791829,0.000988,-0.181799,2,50.0,0.000000,8.716975,0.000161,135.492613,0.217727,0.123618,0.648620,200.84,0.014226,0.99,4.157949,3.424282e-09,149.179095,0.646620,2,-0.206030,1.000000,850090598,0.192295,31.500639,25.73,NaN,200.0,3.813358,0.693333,3.966638,1.281982,6.984628,0.183702,0.029445,0.013855,0.033660,35.0,0.022935,0.971,0.02,8.0,-0.260148,0.218472,0.244552,NaN,-0.152564,135.0,90.0,0.314881,0.076900,0.064436,NaN,0.771675,-30.0,5014,0.50,31.102239,32.879601,31.817430,33.087895,30.247377,NaN,0.0155,False,False,False,False,False,0.071593,False,0.002935,0.000257,0.002868,0.006018,0.003118,60,11,769322820,5,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970056,1249.998752,True
951791844,0.003398,-0.130621,3,70.0,0.000000,4.807868,0.008475,93.539639,0.177306,0.096147,0.618598,235.14,0.004809,0.99,3.271097,6.448362e-03,98.237100,0.375456,3,-0.137353,0.974000,850090604,0.178559,38.571969,29.65,NaN,600.0,0.955155,0.722449,0.960036,1.108425,0.876520,0.112999,0.017166,0.012749,0.013761,35.0,0.773651,0.046,0.16,4.0,0.107106,0.142094,0.135645,-0.191860,0.127236,45.0,30.0,0.106669,0.164823,0.278626,0.505496,0.315617,10.0,4930,0.50,38.229295,36.689798,39.810075,38.408099,38.312633,NaN,0.0315,False,False,False,False,False,0.024339,False,0.002119,0.000381,0.001555,0.005764,0.002682,100,43,769322820,8,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970056,1249.998752,True
951791868,0.001211,-0.117704,5,60.0,-0.343384,4.381891,0.020884,93.768051,0.093410,0.137353,0.606729,173.64,0.009742,0.99,3.244161,9.937440e-03,105.767415,0.462754,5,0.137353,0.970667,850090608,0.178559,36.415146,26.10,NaN,800.0,6.303737,1.565957,2.194863,2.444444,4.075222,0.226879,0.008678,0.024218,0.016777,75.0,0.629035,0.449,0.08,2.0,-0.230550,-0.300395,-0.302802,-0.441741,-0.467807,0.0,0.0,0.025760,0.192805,0.006803,0.005356,0.000061,25.0,5017,0.00,36.962734,35.330917,36.518038,33.968382,37.322128,NaN,0.1415,False,False,False,False,False,-0.051458,False,0.004566,0.008378,0.006339,0.017257,0.006285,120,59,769322820,10,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970056,1249.998752,True
951791891,0.088841,-0.071440,6,100.0,-0.824121,4.853266,0.000517,65.376150,0.150088,0.123618,0.486890,347.24,0.009593,0.99,2.064890,1.339262e-03,62.643555,0.274576,6,-0.137353,0.956790,850090616,0.219765,4.489890,31.97,NaN,100.0,12.704478,1.428333,2.217647,1.782456,1.968569,0.723081,0.042805,0.088331,0.019611,10.0,0.317167,0.009,0.02,15.0,-0.632727,0.793313,0.443038,NaN,0.187500,45.0,90.0,0.268

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))

Total number of low-contamination V1 units: 56


In [ ]:
stim_table = session.get_stimulus_table(['natural_scenes'])

stim_table.head()

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']

presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=V1_units
)

times.head()